<a href="https://colab.research.google.com/github/yepez26/BRFUNK_ismir_2024/blob/main/maritalk_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install maritalk

In [ ]:
import pandas as pd
import numpy as np
import maritalk
import random
import IPython
# from IPython.core.debugger import set_trace


In [ ]:
lyrics = pd.read_excel('/content/drive/MyDrive/UFRGS/mestrado/matitalk/base_estrofes_ext.xlsx')

In [ ]:

musicas_teste = [ 'Quem Vai Patrocinar? (part. MC IG)',
                 'Melanina (part. MC 7 Belo)',
                 'Banco do Carona (part. Don Juan, Marks, Joaozinho VT, Kako, Robs, Gabb e Vine 7)',
                 'Meca Cereja (part. Kayblack e Vulgo FK)',
                 'Melhor Opção (part. MC Kadu e MC Kanhoto)',
                 'Rap do Solitário',
                  'Amor Só de Mãe',
                 'A Favela Venceu',
                 'Obrigado, Mãe (Pt. 2) (part. Dfideliz)',
                 'Hit do Ano - O Peso da Luta (part. MC Leozinho ZS, MC Don Juan, MC IG, MC Menor da VG, MC Neguinho do Kaxeta, MC Marks, MC Kevin, MC Lele JP, MC Ryan SP, MC Vitão do Savoy e MC Kelvinho)',
                 '2023 (part. MC Kadu e MC GP)'
                 ]

# base reduzida com músicas de teste
lyrics_test = lyrics[lyrics['musica'].isin(musicas_teste)].reset_index(drop=True)


In [ ]:
#parameters

num_iter = 20 # numero de vezes que será repetida a interação
num_groups = 20 # numero de grupos diferente que será separada cada interação
num_topicos = 5 # numero de tópicos que será perguntado o modelo
aux_dic = {number: ['topics:'] for number in range(0,num_topicos)} # dicionario auxiliar para montagem de base final
tamanho = len(lyrics_test)

In [ ]:
# definindo os do prompt

model = maritalk.MariTalk(key="")
template1 = "dado os seguintes trechos de música, sugera {} tópicos que descrevem os assuntos abordados: ".format(num_topicos)

In [ ]:
# %xmode Verbose
# %pdb on

lista_dfs = [] # lista onde vão ser salvo os dfs de cada iteração

for iter in range(0,num_iter):

    random.seed(iter)
    df_1_volta = pd.DataFrame() # df com o resultado final de topicos ao longo de 1 iteração
    # definido os trechos de comparação aletóriamente usando a biblioteca random
    list_sample = []
    for i in range(0,tamanho):
      list_sample.append(random.randrange(0,num_groups))

    lyrics_test['sample'] = list_sample

    # fazendo a iteração dentro dos grupos criados
    # df_aux1 = pd.DataFrame()
    for grupo in range(0,num_groups):

        # filtrar musicas do grupo
        lista_trechos = list(lyrics_test[lyrics_test['sample']==grupo]['trecho'].reset_index(drop=True))

        # string com trechos músicas
        string_trechos = ['trecho {}: '.format(i) + lista_trechos[i] for i in range(0,len(lista_trechos))]
        string_trechos = '; '.join(string_trechos) + ' '

        #template com frases do grupo
        template2 = template1 + string_trechos

        # chamda modelo
        answer = model.generate(
            template2,
          do_sample=True,
            # max_tokens=200,
            temperature=0.7,
            top_p=0.95)

        # splitar respostas em lista
        list_answer = answer.split('\n')

        # df auxiliar final
        df_aux = pd.DataFrame()
        df_aux['trechos'] = lista_trechos

        for col in aux_dic.keys():
          df_aux[col] = str(aux_dic[col])

        for i in range(0,num_topicos):
          # concat horizontal two list
          df_aux[i] = df_aux[i]+str(list_answer[i])

        df_1_volta = df_1_volta.append(df_aux)
        # set_trace()

    df_1_volta.to_excel('/content/drive/MyDrive/UFRGS/mestrado/matitalk/base/base_topics_{}.xlsx'.format(iter))
    lista_dfs.append([df_1_volta])
    # set_trace()



